In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation import my_cellpose
from nuclei_segmentation.quality_control.model_tester import ModelTester

GPU activated: False


In [2]:
v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

img_path = (
        v.data_path 
        + 'Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz'
)

Grid search into preprocessing steps 

In [3]:
from itertools import permutations, chain


# Cellpose params
steps_type = ['2D', '3D']
steps_thr = list(np.round(np.linspace(0.2, .8, 8), 1))

# Preprocessing steps
normalization_methods = ['norm_minmax', 'norm_percentile']
pre_steps_top = ['isotropy']
pre_steps_bottom = [
    'bilateral', 
    'anisodiff'
]

pre_steps_permutations = list(chain(*[
    permutations(pre_steps_bottom, r) 
    for r in range(1, len(pre_steps_bottom) + 1)
]))

# Postprocessing steps
post_steps = [
    'remove_small_objects',
    '3d_connected_component_analysis',
    'watershed'
]

post_steps_permutations = list(chain(*[
    permutations(post_steps, r) 
    for r in range(1, len(post_steps) + 1)
])) + [()]

pipelines_dict = {}

# Create pipelines for each type (2D/3D)
for step in steps_type:
    # Apply threshold only for 2D
    if step == '2D':
        for thr in steps_thr:
            for norm_method in normalization_methods:
                for i, pre_steps in enumerate(pre_steps_permutations):
                    for j, post_steps in enumerate(post_steps_permutations):
                        pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                        pipeline = pipeline_pre + list(post_steps)
                        
                        # Include threshold for 2D pipelines
                        name = f"pipeline_{step}_thr_{thr}_{norm_method}_{i}_{j}"
                        pipelines_dict[name] = {
                            'pipeline': pipeline,
                            'type': step,
                            'threshold': thr
                        }
    else:  # For 3D pipelines, ignore thresholds
        for norm_method in normalization_methods:
            for i, pre_steps in enumerate(pre_steps_permutations):
                for j, post_steps in enumerate(post_steps_permutations):
                    pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                    pipeline = pipeline_pre + list(post_steps)
                    
                    # No threshold for 3D pipelines
                    name = f"pipeline_{step}_{norm_method}_{i}_{j}"
                    pipelines_dict[name] = {
                        'pipeline': pipeline,
                        'type': step
                    }

print('Total pipelines:', len(pipelines_dict))

Total pipelines: 896


In [4]:
# Print all pipelines
for name, config in pipelines_dict.items():
    print(name, config)

pipeline_2D_thr_0.2_norm_minmax_0_0 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'remove_small_objects'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_1 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', '3d_connected_component_analysis'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_2 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'watershed'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_3 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'remove_small_objects', '3d_connected_component_analysis'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_4 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', 'remove_small_objects', 'watershed'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_5 {'pipeline': ['isotropy', 'norm_minmax', 'bilateral', '3d_connected_component_analysis', 'remove_small_objects'], 'type': '2D', 'threshold': 0.2}
pipeline_2D_thr_0.2_norm_minmax_0_6

In [6]:
model = my_cellpose.load_model('nuclei')
tester = ModelTester(model)

Loading model: nuclei


In [8]:
bar = LoadingBar(len(pipelines_dict))

for name, config in pipelines_dict.items():
    print(f"\nTesting {name} pipeline")
    
    pipeline = config['pipeline']
    type = config['type']
    thr = config['threshold'] if 'threshold' in config else None
    
    tester.run(
        img_path, pipeline,
        type=type, stitch_threshold=thr,
        test_name=name, 
        verbose=0
    )
    
    bar.update()
    
bar.end()


Testing pipeline_2D_thr_0.2_norm_minmax_0_0 pipeline


100%|██████████| 19/19 [00:00<00:00, 5078.17it/s]

[                                                  ] 0.11%


Testing pipeline_2D_thr_0.2_norm_minmax_0_1 pipeline


100%|██████████| 19/19 [00:00<00:00, 5990.51it/s]

...

100%|██████████| 19/19 [00:00<00:00, 5951.14it/s]


[==========================================        ] 85.60%
Testing pipeline_2D_thr_0.7_norm_percentile_3_15 pipeline


100%|██████████| 19/19 [00:00<00:00, 3522.60it/s]

[==========================================        ] 85.71%


Testing pipeline_3D_norm_minmax_0_0 pipeline
[==========================================        ] 85.83%
Testing pipeline_3D_norm_minmax_0_1 pipeline
[==========================================        ] 85.94%
Testing pipeline_3D_norm_minmax_0_2 pipeline
[===========================================       ] 86.05%
Testing pipeline_3D_norm_minmax_0_3 pipeline
[===========================================       ] 86.16%
Testing pipeline_3D_norm_minmax_0_4 pipeline
[===========================================       ] 86.27%
Testing pipeline_3D_norm_minmax_0_5 pipeline
[===========================================       ] 86.38%
Testing pipeline_3D_norm_minmax_0_6 pipeline
[===========================================       ] 86.50%
Testing pipeline_3D_norm_minmax_0_7 pipeline
[===========================================       ] 86.61%
Testing pipeline_3D_norm_minmax_0_8 pipeline
[===========================================       ] 86.72%
Testing pipeline_3D_norm_minmax_0_9 pipeline
[========